### 기상정보 취합 - 2024

In [20]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta

# API 기본 설정
API_URL = 'http://apis.data.go.kr/1360000/AirInfoService/getAirInfo'
PARAMS = {
    'numOfRows': '100',
    'pageNo': '1',
    'dataType': 'JSON',
    'icaoCode': 'RKSI'  # 인천공항
}
# API 인증키 추가 필요

# 시작 날짜 및 종료 날짜 설정
START_DATE = datetime(2024, 1, 1, 0, 0)
END_DATE = datetime(2024, 12, 31, 23, 0)

# 3시간 간격으로 시간 리스트 생성
def generate_time_list(start, end):
    time_list = []
    current_time = start
    while current_time <= end:
        time_list.append(current_time.strftime('%Y%m%d%H%M'))
        current_time += timedelta(hours=3)
    return time_list

time_points = generate_time_list(START_DATE, END_DATE)

# API 호출 실행
REQUEST_LIMIT = 10000  # 하루 요청 제한
requests_today = 0

# 데이터 저장 리스트
data_list = []

def fetch_weather_data(timestamp):
    global requests_today
    if requests_today >= REQUEST_LIMIT:
        print("일일 트래픽 제한 도달. 종료합니다.")
        return False
    
    PARAMS['fctm'] = timestamp
    try:
        response = requests.get(API_URL, params=PARAMS, timeout=10)
        response.raise_for_status()
        json_data = response.json()
        
        # JSON에서 필요한 데이터 추출
        for item in json_data.get('response', {}).get('body', {}).get('items', {}).get('item', []):
            data_list.append({
                'timestamp': item.get('tmFc', 'N/A'),
                'airport_name': bytes(item.get('airportName', ''), 'utf-8').decode('utf-8', 'ignore'),
                'wind_direction': item.get('wd', 'N/A'),
                'wind_speed': item.get('ws', 'N/A'),
                'temperature': item.get('ta', 'N/A'),
                'pressure': item.get('qnh', 'N/A')
            })
        
        print(f"{timestamp} 데이터 수집 완료.")
        requests_today += 1
    except requests.exceptions.RequestException as e:
        print(f"{timestamp} 데이터 요청 실패: {e}")
    
    return True

# 전체 데이터 수집
for timestamp in time_points:
    if not fetch_weather_data(timestamp):
        break
    time.sleep(1)  # 과부하 방지를 위한 대기

# 데이터 CSV 저장
if data_list:
    df = pd.DataFrame(data_list)
    df.to_csv('weather_data.csv', index=False, encoding='utf-8-sig')
    print("CSV 파일 저장 완료: weather_data.csv")

202401010000 데이터 수집 완료.
202401010300 데이터 수집 완료.
202401010600 데이터 수집 완료.
202401010900 데이터 수집 완료.
202401011200 데이터 수집 완료.
202401011500 데이터 수집 완료.
202401011800 데이터 수집 완료.
202401012100 데이터 수집 완료.
202401020000 데이터 수집 완료.
202401020300 데이터 수집 완료.
202401020600 데이터 수집 완료.
202401020900 데이터 수집 완료.
202401021200 데이터 수집 완료.
202401021500 데이터 수집 완료.
202401021800 데이터 수집 완료.
202401022100 데이터 수집 완료.
202401030000 데이터 수집 완료.
202401030300 데이터 수집 완료.
202401030600 데이터 수집 완료.
202401030900 데이터 수집 완료.
202401031200 데이터 수집 완료.
202401031500 데이터 수집 완료.
202401031800 데이터 수집 완료.
202401032100 데이터 수집 완료.
202401040000 데이터 수집 완료.
202401040300 데이터 수집 완료.
202401040600 데이터 수집 완료.
202401040900 데이터 수집 완료.
202401041200 데이터 수집 완료.
202401041500 데이터 수집 완료.
202401041800 데이터 수집 완료.
202401042100 데이터 수집 완료.
202401050000 데이터 수집 완료.
202401050300 데이터 수집 완료.
202401050600 데이터 수집 완료.
202401050900 데이터 수집 완료.
202401051200 데이터 수집 완료.
202401051500 데이터 수집 완료.
202401051800 데이터 수집 완료.
202401052100 데이터 수집 완료.
202401060000 데이터 수집 완료.
202401060300 데이터

In [25]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
import logging

# API 기본 설정
API_URL = 'http://apis.data.go.kr/1360000/AirInfoService/getAirInfo'
PARAMS = {
    'numOfRows': '100',
    'pageNo': '1',
    'dataType': 'JSON',
    'icaoCode': 'RKSI'  # 인천공항
}
# API 인증키 추가 필요

# 시작 날짜 및 종료 날짜 설정
START_DATE = datetime(2024, 1, 1, 0, 0)
END_DATE = datetime(2024, 12, 31, 23, 0)

# 3시간 간격으로 시간 리스트 생성
def generate_time_list(start, end):
    time_list = []
    current_time = start
    while current_time <= end:
        time_list.append(current_time.strftime('%Y%m%d%H%M'))
        current_time += timedelta(hours=3)
    return time_list

time_points = generate_time_list(START_DATE, END_DATE)

# 이미 존재하는 데이터 파일 확인
def load_existing_data():
    try:
        # CSV 파일에서 데이터 읽어오기
        df = pd.read_csv('weather_data.csv', encoding='utf-8-sig')
        # 'timestamp' 컬럼만 리스트로 반환하고, 중복을 피하기 위해 set으로 반환
        existing_timestamps = set(df['timestamp'].astype(str).tolist())  
        return existing_timestamps
    except FileNotFoundError:
        return set()  # 파일이 없으면 빈 set 반환

# API 호출 실행
REQUEST_LIMIT = 10000  # 하루 요청 제한
requests_today = 0

# 데이터 저장 리스트
data_list = []

# 로그 설정
logging.basicConfig(filename='data_request_errors.log', level=logging.ERROR)

def fetch_weather_data(timestamp):
    global requests_today
    if requests_today >= REQUEST_LIMIT:
        print("일일 트래픽 제한 도달. 종료합니다.")
        return False
    
    PARAMS['fctm'] = timestamp
    retry_count = 3  # 실패 시 재시도 횟수
    for _ in range(retry_count):
        try:
            response = requests.get(API_URL, params=PARAMS, timeout=15)  # 타임아웃 시간 증가
            response.raise_for_status()
            json_data = response.json()
            
            if not json_data or not json_data.get('response', {}).get('body', {}).get('items', {}).get('item', []):
                raise ValueError("빈 응답 또는 잘못된 응답 형식")
            
            # JSON에서 필요한 데이터 추출
            for item in json_data.get('response', {}).get('body', {}).get('items', {}).get('item', []):
                data_list.append({
                    'timestamp': item.get('tmFc', 'N/A'),
                    'airport_name': bytes(item.get('airportName', ''), 'utf-8').decode('utf-8', 'ignore'),
                    'wind_direction': item.get('wd', 'N/A'),
                    'wind_speed': item.get('ws', 'N/A'),
                    'temperature': item.get('ta', 'N/A'),
                    'pressure': item.get('qnh', 'N/A')
                })
            
            print(f"{timestamp} 데이터 수집 완료.")
            requests_today += 1
            return True
        except (requests.exceptions.RequestException, ValueError) as e:
            logging.error(f"{timestamp} - 데이터 요청 실패: {e}")
            print(f"{timestamp} 데이터 요청 실패: {e}")
            time.sleep(5)  # 5초 후 재시도
    return False

# 기존에 수집된 데이터의 타임스탬프 목록 로드
existing_timestamps = load_existing_data()

# 전체 데이터 수집
for timestamp in time_points:
    if timestamp in existing_timestamps:
        print(f"{timestamp} 이미 데이터가 존재합니다. 건너뜁니다.")
        continue  # 이미 수집된 데이터는 건너뛰기
    
    if not fetch_weather_data(timestamp):
        break
    time.sleep(1)  # 과부하 방지를 위한 대기

# 데이터 CSV 저장
if data_list:
    df = pd.DataFrame(data_list)
    df.to_csv('weather_data.csv', index=False, encoding='utf-8-sig', mode='a', header=not bool(existing_timestamps))
    print("CSV 파일 저장 완료: weather_data.csv")


202401010000 이미 데이터가 존재합니다. 건너뜁니다.
202401010300 이미 데이터가 존재합니다. 건너뜁니다.
202401010600 이미 데이터가 존재합니다. 건너뜁니다.
202401010900 이미 데이터가 존재합니다. 건너뜁니다.
202401011200 이미 데이터가 존재합니다. 건너뜁니다.
202401011500 이미 데이터가 존재합니다. 건너뜁니다.
202401011800 이미 데이터가 존재합니다. 건너뜁니다.
202401012100 이미 데이터가 존재합니다. 건너뜁니다.
202401020000 이미 데이터가 존재합니다. 건너뜁니다.
202401020300 이미 데이터가 존재합니다. 건너뜁니다.
202401020600 이미 데이터가 존재합니다. 건너뜁니다.
202401020900 이미 데이터가 존재합니다. 건너뜁니다.
202401021200 이미 데이터가 존재합니다. 건너뜁니다.
202401021500 이미 데이터가 존재합니다. 건너뜁니다.
202401021800 이미 데이터가 존재합니다. 건너뜁니다.
202401022100 이미 데이터가 존재합니다. 건너뜁니다.
202401030000 이미 데이터가 존재합니다. 건너뜁니다.
202401030300 이미 데이터가 존재합니다. 건너뜁니다.
202401030600 이미 데이터가 존재합니다. 건너뜁니다.
202401030900 이미 데이터가 존재합니다. 건너뜁니다.
202401031200 이미 데이터가 존재합니다. 건너뜁니다.
202401031500 이미 데이터가 존재합니다. 건너뜁니다.
202401031800 이미 데이터가 존재합니다. 건너뜁니다.
202401032100 이미 데이터가 존재합니다. 건너뜁니다.
202401040000 이미 데이터가 존재합니다. 건너뜁니다.
202401040300 이미 데이터가 존재합니다. 건너뜁니다.
202401040600 이미 데이터가 존재합니다. 건너뜁니다.
202401040900 이미 데이터가 존재합니다. 건너뜁니다.
202401041200 이미 데이터가

### 데이터 종합 끝